# Preparation

In [1]:
import sys
!{sys.executable} -m pip install --user pandas numpy sklearn nltk

In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Define constants
tfidf = TfidfVectorizer(stop_words='english')

In [4]:
# By https://ru.stackoverflow.com/questions/995616/Как-сделать-tf-idf-для-русских-текстов

#import nltk
#from nltk.corpus import stopwords as nltk_stopwords

#nltk.download('stopwords')
#stopwords = set(nltk_stopwords.words('russian') )
#tfidf = TfidfVectorizer(stop_words=stopwords)

In [5]:
# Minimal example:
articles = pd.read_csv('data/TF-IDF/TF-IDF-min.csv.Ru', index_col='№')
# articles = pd.read_csv('data/TF-IDF/TF-IDF-min.csv.En', index_col='№')
# Full real file:
# articles = pd.read_csv('RDC-135_articles_golden_set_mapping.csv', index_col='№')
articles

,Article
№,
1,"Раз, два, три, четыре"
2,вышел зайчик погулять
3,вдруг охотник выбегает
4,прямо в зайчик стреляет
5,пуляет прямо в зайчик


In [6]:
tfidf_matrix = tfidf.fit_transform(articles['Article'])
tfidf_matrix.toarray() # or res.todense()

array([[0.        , 0.        , 0.        , 0.5       , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.5       ,
        0.        , 0.5       , 0.5       ],
       [0.        , 0.        , 0.63907044, 0.        , 0.42799292,
        0.        , 0.63907044, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.57735027, 0.57735027, 0.        , 0.        , 0.        ,
        0.57735027, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.4622077 ,
        0.        , 0.        , 0.55681615, 0.        , 0.        ,
        0.69015927, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.4622077 ,
        0.        , 0.        , 0.55681615, 0.69015927, 0.        ,
        0.        , 0.        , 0.        ]])

In [7]:
tfidf_matrix.shape

(5, 13)

In [8]:
tfidf.stop_words

'english'

# Вычисление

## Самые популярные слова

In [9]:
# By https://www.rupython.com/sklearn-tfidf-transformer-tf-idf-33655.html
# Вышеупомянутый X имеет значения TF-IDF всех документов в корпусе. Это большая разреженная матрица.
# Теперь,
tfidf.get_feature_names()
# это дает вам список всех токенов или n-граммов или слов. Для первого документа в вашем корпусе, 

['вдруг',
 'выбегает',
 'вышел',
 'два',
 'зайчик',
 'охотник',
 'погулять',
 'прямо',
 'пуляет',
 'раз',
 'стреляет',
 'три',
 'четыре']

In [10]:
# By https://stackoverflow.com/questions/37593293/what-is-the-simplest-way-to-get-tfidf-with-pandas-dataframe#comment72191707_37593408
# v.get_feature_names() will give you the list of feature names.
# v.vocabulary_ will give you a dict with feature names as keys and their index in the matrix produced as values.
tfidf.vocabulary_

{'раз': 9,
 'два': 3,
 'три': 11,
 'четыре': 12,
 'вышел': 2,
 'зайчик': 4,
 'погулять': 6,
 'вдруг': 0,
 'охотник': 5,
 'выбегает': 1,
 'прямо': 7,
 'стреляет': 10,
 'пуляет': 8}

In [11]:
# By https://www.rupython.com/sklearn-tfidf-transformer-tf-idf-33655.html
# это дает вам список всех токенов или n-граммов или слов. Для первого документа в вашем корпусе, 
# Позволяет распечатать их:
def print_word_ratings_by_document(doc=0):
    feature_names = tfidf.get_feature_names()
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = zip(feature_index, [tfidf_matrix[doc, x] for x in feature_index]) 
    for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
        print (w, s)

for i, _ in articles.iterrows():
    print("### doc {}: ###".format(i-1))
    print_word_ratings_by_document(i-1)

### doc 0: ###
четыре 0.5
три 0.5
два 0.5
раз 0.5
### doc 1: ###
погулять 0.6390704413963749
зайчик 0.42799292268317357
вышел 0.6390704413963749
### doc 2: ###
выбегает 0.5773502691896258
охотник 0.5773502691896258
вдруг 0.5773502691896258
### doc 3: ###
стреляет 0.6901592662889633
прямо 0.5568161504458247
зайчик 0.46220770413113277
### doc 4: ###
пуляет 0.6901592662889633
прямо 0.5568161504458247
зайчик 0.46220770413113277


In [12]:
# https://ru.stackoverflow.com/questions/772859/tfidfvectorizer/773018#773018
# Топ 10 самых популярных слов:
N=10
idx = np.ravel(tfidf_matrix.sum(axis=0).argsort(axis=1))[::-1][:N]
top_10_words = np.array(tfidf.get_feature_names())[idx].tolist()
top_10_words

['зайчик',
 'прямо',
 'стреляет',
 'пуляет',
 'погулять',
 'вышел',
 'охотник',
 'выбегает',
 'вдруг',
 'четыре']

### Разбор

In [13]:
tfidf_matrix.shape

(5, 13)

In [14]:
tfidf_matrix.todense()

matrix([[0.        , 0.        , 0.        , 0.5       , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.5       ,
         0.        , 0.5       , 0.5       ],
        [0.        , 0.        , 0.63907044, 0.        , 0.42799292,
         0.        , 0.63907044, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.57735027, 0.57735027, 0.        , 0.        , 0.        ,
         0.57735027, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.4622077 ,
         0.        , 0.        , 0.55681615, 0.        , 0.        ,
         0.69015927, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.4622077 ,
         0.        , 0.        , 0.55681615, 0.69015927, 0.        ,
         0.        , 0.        , 0.        ]])

In [15]:
# Doc: https://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html#numpy.sum
tfidf_matrix.sum(axis=0)  # Сумма по столбцам

matrix([[0.57735027, 0.57735027, 0.63907044, 0.5       , 1.35240833,
         0.57735027, 0.63907044, 1.1136323 , 0.69015927, 0.5       ,
         0.69015927, 0.5       , 0.5       ]])

In [16]:
# Doc: https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html
tfidf_matrix.sum(axis=0).argsort(axis=1)  # Возвращает *индексы*, по возрастанию значений элементов

matrix([[ 3,  9, 11, 12,  0,  1,  5,  2,  6,  8, 10,  7,  4]])

In [17]:
# Doc: https://docs.scipy.org/doc/numpy/reference/generated/numpy.ravel.html
np.ravel(tfidf_matrix.sum(axis=0).argsort(axis=1))

array([ 3,  9, 11, 12,  0,  1,  5,  2,  6,  8, 10,  7,  4])

In [18]:
np.ravel(tfidf_matrix.sum(axis=0).argsort(axis=1))[::-1]  # Reverse list (DESC)

array([ 4,  7, 10,  8,  6,  2,  5,  1,  0, 12, 11,  9,  3])

In [19]:
np.ravel(tfidf_matrix.sum(axis=0).argsort(axis=1))[::-1][:N] # Take top N elements

array([ 4,  7, 10,  8,  6,  2,  5,  1,  0, 12])

In [20]:
np.ravel(tfidf_matrix.sum(axis=0).argsort(axis=1))[-N:][::-1]

array([ 4,  7, 10,  8,  6,  2,  5,  1,  0, 12])

## Поиск самого похожего документа

In [21]:
similarities = (tfidf_matrix * tfidf_matrix.T).A[-1,:-1]
similarities

array([0.        , 0.19782163, 0.        , 0.52368019])

In [22]:
# Doc: https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html
#  numpy.argmax(a, axis=None, out=None)
# Returns the indices of the maximum values along an axis.
max_sim_position = np.argmax(similarities)
max_sim_position

3

In [23]:
max_sim = max(similarities)
max_sim

0.52368018715548

In [24]:
similarities[max_sim_position]

0.52368018715548

### Разбор

In [25]:
tfidf_matrix.todense()

matrix([[0.        , 0.        , 0.        , 0.5       , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.5       ,
         0.        , 0.5       , 0.5       ],
        [0.        , 0.        , 0.63907044, 0.        , 0.42799292,
         0.        , 0.63907044, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.57735027, 0.57735027, 0.        , 0.        , 0.        ,
         0.57735027, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.4622077 ,
         0.        , 0.        , 0.55681615, 0.        , 0.        ,
         0.69015927, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.4622077 ,
         0.        , 0.        , 0.55681615, 0.69015927, 0.        ,
         0.        , 0.        , 0.        ]])

In [26]:
tfidf_matrix.shape

(5, 13)

In [27]:
type(tfidf_matrix)

scipy.sparse.csr.csr_matrix

In [52]:
mul = tfidf_matrix * tfidf_matrix.T
mul.todense()

matrix([[1.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.19782163, 0.19782163],
        [0.        , 0.        , 1.        , 0.        , 0.        ],
        [0.        , 0.19782163, 0.        , 1.        , 0.52368019],
        [0.        , 0.19782163, 0.        , 0.52368019, 1.        ]])

In [53]:
mul.todense().shape

(5, 5)

In [54]:
scores = mul.A
scores

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.19782163, 0.19782163],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.19782163, 0.        , 1.        , 0.52368019],
       [0.        , 0.19782163, 0.        , 0.52368019, 1.        ]])

In [32]:
# We will search maximum, so do not willing match to himself:
np.fill_diagonal(scores, -1)
scores

array([[-1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.19782163,  0.19782163],
       [ 0.        ,  0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.19782163,  0.        , -1.        ,  0.52368019],
       [ 0.        ,  0.19782163,  0.        ,  0.52368019, -1.        ]])

In [33]:
type(scores)

numpy.ndarray

In [34]:
scores_df = pd.DataFrame(scores, index=articles.index)
scores_df

,0,1,2,3,4
№,,,,,
1,-1.0,0.000000,0.0,0.000000,0.000000
2,0.0,-1.000000,0.0,0.197822,0.197822
3,0.0,0.000000,-1.0,0.000000,0.000000
4,0.0,0.197822,0.0,-1.000000,0.523680
5,0.0,0.197822,0.0,0.523680,-1.000000


In [59]:
# By https://gist.github.com/RZachLamberty/1ed47cd0e2d0d968f7cdbd3d53a50f4c
# you can calculate cosine similarity easily given this
(tfidf_matrix @ tfidf_matrix.T).todense()

matrix([[1.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.19782163, 0.19782163],
        [0.        , 0.        , 1.        , 0.        , 0.        ],
        [0.        , 0.19782163, 0.        , 1.        , 0.52368019],
        [0.        , 0.19782163, 0.        , 0.52368019, 1.        ]])

In [56]:
# Doc: https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.A.html
# Return self as an ndarray object.
# Equivalent to np.asarray(self)
mul.A

array([[1.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.19782163, 0.19782163],
       [0.        , 0.        , 1.        , 0.        , 0.        ],
       [0.        , 0.19782163, 0.        , 1.        , 0.52368019],
       [0.        , 0.19782163, 0.        , 0.52368019, 1.        ]])

In [57]:
mul.A[-1,:-1] # Последняя строка

array([0.        , 0.19782163, 0.        , 0.52368019])

In [38]:
mul.A[-1,0:] # Последняя строка

array([0.        , 0.19782163, 0.        , 0.52368019, 1.        ])

In [39]:
mul.A[-1,:-1] # Последняя строка без последнего элемента.
# Матрица диагональная (остальные не имеют значения, зеркально повторяются), содержит веса всех со всеми.
# Последний элемент, выкидывается потому что это матч самого к себе, если мы рассматриваем максимальный дубль
# для последнего в наборе документа (так было в функции get_similarities из GOJI)

# Получается что реально нужно выкидывать по индексу того, для которого ищется сравнение!

array([0.        , 0.19782163, 0.        , 0.52368019])

### Обобщение варианта - без пересчёта матрицы каждый раз!

In [40]:
# Получается что реально нужно выкидывать по индексу того, для которого ищется сравнение!
forDocNo=1 # Expect match doc 3<>4

In [41]:
scores

array([[-1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.19782163,  0.19782163],
       [ 0.        ,  0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.19782163,  0.        , -1.        ,  0.52368019],
       [ 0.        ,  0.19782163,  0.        ,  0.52368019, -1.        ]])

In [42]:
s = scores[forDocNo]
max_sim_position = np.argmax(s)

(max_sim_position, s[max_sim_position])

(3, 0.19782162617776308)

### Для каждого документа, в DataFrame add "Max TF/IDF DUP score" and "Max DUP score docId"

In [43]:
articles

# Should be matched:
# id № Dup№
#  0 1 -
#  1 2 (4 and 5)
#  2 3 -
#  3 4 5
#  4 5 4

,Article
№,
1,"Раз, два, три, четыре"
2,вышел зайчик погулять
3,вдруг охотник выбегает
4,прямо в зайчик стреляет
5,пуляет прямо в зайчик


In [44]:
scores

array([[-1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.        ,  0.19782163,  0.19782163],
       [ 0.        ,  0.        , -1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.19782163,  0.        , -1.        ,  0.52368019],
       [ 0.        ,  0.19782163,  0.        ,  0.52368019, -1.        ]])

In [45]:
scores_df

,0,1,2,3,4
№,,,,,
1,-1.0,0.000000,0.0,0.000000,0.000000
2,0.0,-1.000000,0.0,0.197822,0.197822
3,0.0,0.000000,-1.0,0.000000,0.000000
4,0.0,0.197822,0.0,-1.000000,0.523680
5,0.0,0.197822,0.0,0.523680,-1.000000


In [46]:
scores_df.loc[3]

0    0.0
1    0.0
2   -1.0
3    0.0
4    0.0
Name: 3, dtype: float64

In [47]:
scores_df.iloc[3]

0    0.000000
1    0.197822
2    0.000000
3   -1.000000
4    0.523680
Name: 4, dtype: float64

In [48]:
# By https://stackoverflow.com/questions/26658240/getting-the-index-of-a-row-in-a-pandas-apply-function/48819898#48819898
# index available as row.name
def most_similar(row):
#     print('====')
#     print('row.name={}; scores_df.loc[row.name].idxmax()={}; scores_df.iloc[scores_df.loc[row.name].idxmax()].name={}'.format(
#         row.name
#         ,scores_df.loc[row.name].idxmax()
#         ,scores_df.iloc[scores_df.loc[row.name].idxmax()].name
#         )
#     )
#     print('row.loc={}; row.iloc={}'.format(row.loc, row.iloc))
    max_similar_doc = scores_df.iloc[scores_df.loc[row.name].idxmax()].name # Array index (.iloc) into DataFrame index (.name)
    max_similar_score = scores_df.loc[row.name].max()
    return ((max_similar_doc if max_similar_score > 0 else -1), max_similar_score)

# articles['Max DUP score docId'] = articles.apply(lambda i: np.argmax(scores), axis=1)
# articles['Max DUP score docId'] = articles.apply(lambda i: type(i.index), axis=1)
articles[['Max DUP score docId', 'Max TF/IDF DUP score']] = articles.apply(most_similar, axis=1, result_type='expand')
articles['Max DUP score docId'] = articles['Max DUP score docId'].astype('int32')
articles

,Article,Max DUP score docId,Max TF/IDF DUP score
№,,,
1,"Раз, два, три, четыре",-1,0.000000
2,вышел зайчик погулять,4,0.197822
3,вдруг охотник выбегает,-1,0.000000
4,прямо в зайчик стреляет,5,0.523680
5,пуляет прямо в зайчик,4,0.523680


In [49]:
( scores_df[3], scores_df.loc[3] )

(№
 1    0.000000
 2    0.197822
 3    0.000000
 4   -1.000000
 5    0.523680
 Name: 3, dtype: float64,
 0    0.0
 1    0.0
 2   -1.0
 3    0.0
 4    0.0
 Name: 3, dtype: float64)

# Python constructions
## Apply with 2 columns at once

By https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns/52363890#52363890
+By https://stackoverflow.com/questions/16236684/apply-pandas-function-to-column-to-create-multiple-new-columns/52363890#comment106834440_16242202 for column naming

In [50]:
# articles[['a', 'b']] = articles.apply(lambda i: [1, 2], axis=1, result_type='expand')
# articles

In [51]:
def dump_xml_file(row):
    print(type(row))
    print(row['File name'])
    with open('articles/' + row['File name'].replace('.xml', '.txt'), 'w+') as file:
        file.write(row['Article'])

articles.apply(dump_xml_file, axis=1)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


KeyError: ('File name', 'occurred at index 1')

# Docs and links

* [sklearn: TFIDF Transformer: Как получить значения tf-idf данных слов в документе](https://www.rupython.com/sklearn-tfidf-transformer-tf-idf-33655.html)
* [SO question: Имеется текст, надо вычислить TF-IDF-признаки по имеющимся тексту. Нашел 10 минимальных весов. Требуется найти 10 слов соответствующих абсолютному значению весов. Как можно это сделать?введите сюда описание изображения](https://ru.stackoverflow.com/questions/772859/tfidfvectorizer/773018#773018). С разбором что к чему.